# Build an Agent

## Define tools

In [31]:
pip install --quiet langchain-anthropic langgraph


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
from dotenv import load_dotenv
load_dotenv()

True

In [33]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.msn.com/en-us/weather/topstories/june-26-2024-san-francisco-bay-area-weather-forecast/vi-BB1oVEx8', 'content': 'June 26, 2024 San Francisco Bay Area weather forecast. Posted: June 26, 2024 | Last updated: June 26, 2024. KRON4 Meteorologist John Shrable reports. <a ...'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719415255, 'localtime': '2024-06-26 8:20'}, 'current': {'last_updated_epoch': 1719414900, 'last_updated': '2024-06-26 08:15', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 210, 'wind_dir': 'SSW', 'pressure_mb': 1018.0, 'pressure_in': 30.05, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 89, 'cloud': 100, 'feelslik

## Using Language Models

In [34]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [35]:
model_with_tools = model.bind_tools(tools)

In [36]:
from langchain_core.messages import HumanMessage
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [37]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Sorry, I can't help with that. My capabilities are currently limited to providing information based on pre-existing data.
ToolCalls: []


## Create the agent

In [38]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

In [39]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='77797600-75ee-49c1-b074-c11bd667f69b'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c2a1f806-fe4e-424b-9605-a7deca8381b6-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [40]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='43567aaf-db1f-49f0-a369-6138819fa349'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Zf452z4Z0VmmKMICpt3scPbi', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ab910465-8454-4c99-9bc8-fce8a4b162e3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_Zf452z4Z0VmmKMICpt3scPbi'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111}),
 ToolMessage(content='[{"url": "https://www.wunderground.com/hourly/us/ca/san-francisco/date/2024-6-26", "content": "Current Weather for Popular Cities . San Francisco, CA 58 \\u00b

## Streaming Messages

In [41]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dpnYZIgBGSgVpZEyW7A0wz0p', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7642ab0f-f048-402c-86e1-04d933c2137a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_dpnYZIgBGSgVpZEyW7A0wz0p'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.wunderground.com/hourly/us/ca/san-francisco/date/2024-6-26", "content": "Current Weather for Popular Cities . San Francisco, CA 58 \\u00b0 F Fair; Manhattan, NY warning 73 \\u00b

## Streaming tokens

In [42]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.wunderground.com/hourly/us/ca/san-francisco/94129/date/2024-06-26', 'content': 'San Francisco Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the San Francisco area.'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719415255, 'localtime': '2024-06-26 8:20'}, 'current': {'last_updated_epoch': 1719414900, 'last_updated': '2024-06-26 08:15', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degr

## Adding in memory

In [43]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! It's nice to meet you. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 90, 'total_tokens': 108}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-038e8e82-0c5b-452f-83aa-e12f4c8490e4-0', usage_metadata={'input_tokens': 90, 'output_tokens': 18, 'total_tokens': 108})]}}
----


In [44]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WgrI4iyAEh49Zhz26QeQJPVq', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 123, 'total_tokens': 146}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5f646168-4820-4673-a48b-cf4ef870d3c9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_WgrI4iyAEh49Zhz26QeQJPVq'}], usage_metadata={'input_tokens': 123, 'output_tokens': 23, 'total_tokens': 146})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.

In [45]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm sorry, I can't assist with that.", response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 89, 'total_tokens': 101}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-02afa43a-9b29-4537-af63-018dc4b728fe-0', usage_metadata={'input_tokens': 89, 'output_tokens': 12, 'total_tokens': 101})]}}
----
